In [1]:
# -*- coding: utf-8 -*-

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import folium

     |████████████████████████████████| 972kB 8.8MB/s 
     |████████████████████████████████| 14.8MB 336kB/s 
     |████████████████████████████████| 6.5MB 38.9MB/s 


# 1. 전유부

In [4]:
전유부 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/1.세종시_전유부.csv', encoding='CP949')
전유부.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,특수지명,블록,로트,동명칭,호명칭,층구분코드명,층번호
0,1,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,616,지상,6
1,2,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,624,지상,6


### 변수 삭제

- 대지위치 : 시군구코드, 법정동코드, 번, 지 로 알 수 있음
- 대장구분코드명 : 모두 '집합'으로 같음
- 건물명 : 대지위치와 같은 이유
- 동명칭, 호명칭, 층번호 : 필요없음
- 층구분코드명 : 필요없을듯

In [6]:
전유부 = 전유부.drop(['대지위치','대장구분코드명','건물명','동명칭','호명칭','층구분코드명','층번호'], axis=1)
전유부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,특수지명,블록,로트
0,1,36110,10100,1,4121,1002,NaN,NaN,NaN
1,2,36110,10100,1,4121,1002,NaN,NaN,NaN


### 결측값 처리

In [7]:
전유부.isnull().sum()

순번            0
시군구코드         0
법정동코드         0
대지구분코드        0
번             0
지             0
특수지명      15033
블록        15212
로트        38935
dtype: int64

In [8]:
전유부['특수지명'].unique()

array([nan, '4-1생활권 C1-6블럭', '4-1생활권', '행복도시 4-1생활권', '행정중심복합도시 4-1생활권',
       '행정중심복합도시4-1생활권', '3-3 생활권', '3-3생활권 C13-2', '3-3생활권',
       '세종특별자치시 3-3생활권', '행복중심복합도시3-3생활권', '행정중심복합도시 3-3생활권', '3-2생활권',
       '행정중심복합도시 3-2생활권', '행정중심복합도시3-2생활권', '행정중심복합도시', '3-1생활권',
       '행정중심복합도시 3-1생활권', '행정중심복합도시3-1생활권', '2-3생활권'], dtype=object)

In [9]:
전유부['블록'].unique()

array([nan, 'C 3-1', 'C 4-', 'C1-1BL', 'C1-4', 'C4-7블록', 'L2블록', 'M1블록',
       'M3블록', 'c4-6', 'C2-1', 'C2-2블록', 'L4블록', 'M2블록', 'L3블록', 'C14',
       'C10-2', 'C15', 'C16-1', 'C16-2BL', 'C5-4BL', 'C6', 'C8-1',
       'C8-2블록', 'C5-3블록', 'C12', 'C13-1', 'C13-3블럭', 'C16', 'M4블록',
       'M6블록', 'H1블록', 'M1', 'M3블럭', 'L1블럭', 'M5블록', '3-2생활권', 'M2BL',
       'M3BL', 'M4블럭', 'RC1-3'], dtype=object)

In [10]:
전유부['로트'].unique()

array([nan, '5', '1', '2', '-1', '-4', 'M4블록'], dtype=object)

- 블록 : 결측치가 많고, 특수지명과 일부 겹치는 것으로 보여 삭제
- 로트 : 대부분이 결측치

In [12]:
전유부 = 전유부.drop(['블록','로트'], axis=1)
전유부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,특수지명
0,1,36110,10100,1,4121,1002,NaN
1,2,36110,10100,1,4121,1002,NaN
